In [1]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import time
import importlib
from pathlib import Path
from datetime import datetime

from numba import jit
import numpy as np
import pandas as pd

from neuropy import core

from pyphocorehelpers.preferences_helpers import set_pho_preferences
set_pho_preferences() # Set preferences
# from pyphoplacecellanalysis.General.SpikesDataframeWindow import SpikesDataframeWindow, SpikesWindowOwningMixin
# from pyphoplacecellanalysis.General.DataSeriesToSpatial import DataSeriesToSpatial
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster_Vedo import Spike3DRaster_Vedo

%gui qt5
# curr_epoch_name = 'maze1'
# curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
# curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
# curr_spikes_df = curr_sess.spikes_df

from vedo import show

def trap_exc_during_debug(*args):
    # when app raises uncaught exception, print info
    print(args)


# install exception hook: without this, uncaught exception would cause application to exit
sys.excepthook = trap_exc_during_debug



In [2]:
# Load the saved .h5 spikes dataframe for testing:
finalized_spike_df_cache_file='./pipeline_cache_store.h5'
desired_spikes_df_key = '/filtered_sessions/maze1/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
curr_spikes_df

print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}")

aclu:   	[ 2  3  4  5  7  8  9 10 11 12 14 17 18 21 22 23 24 25 26 27 28 29 33 34 38 39 42 44 45 46 47 48 53 55 57 58 61 62 63 64]
unit_id:	[ 0  1  2  3  5  6  7  8  9 10 12 15 16 19 20 21 22 23 24 25 26 27 31 32 36 37 40 42 43 44 45 46 51 53 55 56 59 60 61 62]


In [3]:
desired_spikes_df_key = '/sess/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
curr_spikes_df

print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}")

aclu:   	[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65]
unit_id:	[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


In [4]:
desired_spikes_df_key = '/filtered_sessions/maze2/spikes_df'
curr_spikes_df = pd.read_hdf(finalized_spike_df_cache_file, key=desired_spikes_df_key)
print(f"aclu:   \t{np.unique(curr_spikes_df['aclu'])}")
print(f"unit_id:\t{np.unique(curr_spikes_df['unit_id'])}") # note unit_id can be missing indicies because each spike might not have an entry in it

aclu:   	[ 2  3  4  5  7  8  9 10 11 12 14 17 18 21 22 23 24 25 26 27 28 29 33 34 38 39 42 44 45 46 47 48 53 55 57 58 61 62 63 64]
unit_id:	[ 0  1  2  3  5  6  7  8  9 10 12 15 16 19 20 21 22 23 24 25 26 27 31 32 36 37 40 42 43 44 45 46 51 53 55 56 59 60 61 62]


In [5]:

def add_filtered_session_cell_idx_column(spikes_df):
    """ 
    
    """
    if 'filtered_session_cell_idx' not in spikes_df.columns:
        # self.spikes_df['cell_idx'] = self.spikes_df['unit_id'].copy() # TODO: this is bad! The self.get_neuron_id_and_idx(...) function doesn't work!
        # note that this is very slow, but works:
        print(f'cell_idx column missing. rebuilding (this might take a minute or two)...')
        included_cell_INDEXES = np.array([self.get_neuron_id_and_idx(neuron_id=an_included_cell_ID)[0] for an_included_cell_ID in spikes_df['aclu'].to_numpy()]) # get the indexes from the cellIDs
        spikes_df['filtered_session_cell_idx'] = included_cell_INDEXES.copy()

# Vedo 3D Spike Train Visualization 

In [6]:
# Build the 3D Raster object:
spike_raster_plt = Spike3DRaster_Vedo(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)


cell_idx column missing. rebuilding (this might take a minute or two)...
visualization_raster_y_location column missing. rebuilding (this might take a minute or two)...


In [7]:
spike_raster_plt._update_plots()
axes = dict(xtitle='window time', ytitle='cell_id', ztitle="")
# plt = show(spike_raster_plt.glyph, __doc__, axes=axes, interactive=False, bg='k8', backend='ipyvtk') # plt: <vedo.plotter.Plotter at 0x22eb25de370>
# plt = show(spike_raster_plt.glyph, __doc__, axes=axes, interactive=True, bg='k8', backend='ipyvtk') # crashes when interactive=True

Spike3DRaster_Vedo._update_plots()


# Pyqtgraph 3D Spike Train Visualization

In [ ]:
spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)
# spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=0.2, window_start_time=30.0)

In [ ]:
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')
# spike_raster_plt.spikes_window.active_window_start_time = 50.0
spike_raster_plt.spikes_window.update_window_start(90.0)
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')